In [1]:
# -*- coding: utf-8 -*-
"""lda-himank.ipynb
Original file is located at
    https://colab.research.google.com/drive/12zx-jsZowxlpoN5yMB7CbRrZWVJVBfMu
"""

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import re
import numpy as np
import pandas as pd
from pprint import pprint
import os
from pathlib import Path
import json
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet as wn
import csv
import random
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.test.utils import datapath
import spacy
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import json
import random


[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
with open('sampled_te_docs_10k.json', 'r') as outfile:  
    test_docs = json.load(outfile)
with open('sampled_tr_docs_10k.json', 'r') as outfile:  
    train_docs = json.load(outfile)

In [3]:
with open('sampled_tr_docs_10k_not_shuffled.json', 'r') as outfile:  
    train_docs_unshuffled = json.load(outfile)
with open('sampled_tr_docs_10k_trustworthy.json', 'r') as outfile:  
    train_docs_trustworthy = json.load(outfile)

In [4]:
def clean(doc):
    deacc = gensim.utils.simple_preprocess(doc, deacc=True, min_len=4)
    stop_free = [i for i in deacc if i not in stop]
    normalized = [lemma.lemmatize(word) for word in stop_free]
    return normalized

stop = set(stopwords.words('english'))
lemma = WordNetLemmatizer()

In [5]:
## LOAD dict
dictionary = corpora.Dictionary.load_from_text('dict_10k')

In [6]:
with open('doc_term_matrix.json', 'r') as outfile:
    doc_term_matrix = json.load(outfile)

In [12]:
###### LOAD MODEL
Lda = gensim.models.wrappers.LdaMallet
ldamodel_10k = Lda.load("model_10k_5000_alpha_005")

In [8]:
# len(test_docs)
len(train_docs)

70000

In [9]:
DOC_NUM_FROM_TESTING = 16000  # range is 0 - 184706
DOC_NUM_FROM_TRAINING = 500   # range is 0 - 70000

In [ ]:
print(test_docs[DOC_NUM_FROM_TESTING])

In [ ]:
tops = ldamodel_10k[test_doc_term_matrix[DOC_NUM_FROM_TESTING]]
tops = sorted(tops, key=lambda a:a[1], reverse=True)
for top in tops[:10]:
    print(top[1], ldamodel_10k.show_topic(top[0]))
    print("\n")
    

In [14]:
doc_term_matrix_unshuffled = [dictionary.doc2bow(doc) for doc in [clean(d) for d in train_docs_unshuffled]]
#doc_term_matrix_trustworthy = [dictionary.doc2bow(doc) for doc in [clean(d) for d in train_docs_trustworthy]]

In [15]:
print(train_docs[DOC_NUM_FROM_TRAINING])

The Washington Free Beacon has a report on Israeli Army prepared for possible cross border attacks on resort area of Eilat by Islamic State affiliate Ansar Beit al Maqdis that has undertaken murderous attacks on Egyptian security forces.

Their tactics could include simultaneous penetration of the border fence at a number of points by large numbers of terrorists, cross border rockets, and the use of hang gliders and speedboats, according to Israel Radio.

“The scenario we are preparing for,” said IDF Col. Hen, in a blunt briefing, “is a terror threat directed against our forces and civilians in the Eilat area. The scenario is a multi-pronged attack, not just at one site, and we will likely have to deal with this challenge without prior warning.” Its members have inflicted heavy casualties on Egyptian troops who have been unable to track down the group’s leaders, whose lairs are believed to be in the mountainous central part of Sinai. In the past year alone, some 350 Egyptian soldiers, 

In [16]:
print(train_docs_unshuffled[6855])

According to new reporting this month, Donald Trump allegedly had a consensual affair with a paid pornographic actress who went by the stage name Stormy Daniels in 2006, one year after marrying his third wife and just months after his son Barron Trump was born. Daniels consented to the release of an extensive 2011 interview in which she described the affair in explicit detail. The Wall Street Journal revealed that Michael Cohen, an attorney working for Donald Trump’s 2016 campaign, paid Daniels $130,000 to keep quiet about the affair. What’s more, he did so through a Delaware LLC to which Cohen was attached as the “authorized person” rather than a third party with some distance from the future president. This thing is pretty airtight.

What’s interesting is that, so far, Trump’s defenders are not following the president’s lead and denying that this affair occurred. As an entertainer and publicity seeker, Donald Trump made a virtue of his penchant for adultery. Jaws around the nation di

In [46]:
tops = ldamodel_10k[doc_term_matrix_unshuffled[1480]]
tops = sorted(tops, key=lambda a:a[1], reverse=True)
for top in tops[:10]:
    print(top[0])
    print(top[1], ldamodel_10k.show_topic(top[0]))
    print("\n")

42
0.720873470345299 [('photo', 0.0515457446252027), ('image', 0.03772558455824659), ('castro', 0.02398911963174138), ('cuban', 0.020850551864832347), ('cuba', 0.019197572840926925), ('playing', 0.01511743474394518), ('fidel', 0.011790552911021605), ('le', 0.010556049589370717), ('fullscreen', 0.010074802531777999), ('getty', 0.008850761102683475)]


38
0.07306889487306262 [('european', 0.024984678902041736), ('europe', 0.022081734025739444), ('country', 0.0175950714446989), ('british', 0.017172531690481566), ('britain', 0.01669515853304519), ('france', 0.01526626455504306), ('french', 0.015111440828306937), ('germany', 0.01168274037996323), ('london', 0.011234396671289875), ('minister', 0.011069896461632745)]


79
0.06809122166047965 [('bill', 0.038560688230836665), ('house', 0.03285886216067634), ('senate', 0.02905516841745539), ('republican', 0.028434839457379315), ('congress', 0.024445418480602648), ('senator', 0.017751808983254833), ('democrat', 0.01741378541818344), ('vote', 0.01

In [13]:
##SHOULD THIS BE doc_term_matrix, you had test_doc_term_matrix??
tops = ldamodel_10k[doc_term_matrix[DOC_NUM_FROM_TRAINING]]
tops = sorted(tops, key=lambda a:a[1], reverse=True)
for top in tops[:10]:
    print(top[1], ldamodel_10k.show_topic(top[0]))
    print("\n")

0.37255667515990315 [('military', 0.06252925938641116), ('force', 0.036800505602197185), ('army', 0.021121375737336537), ('defense', 0.017610249367997252), ('troop', 0.015803969913548266), ('afghanistan', 0.015534783558565588), ('soldier', 0.013962579195405887), ('security', 0.011153678099934458), ('operation', 0.009347398645485472), ('iraq', 0.008805124683998626)]


0.18203328743239447 [('israel', 0.09830691830457224), ('israeli', 0.05242818414055576), ('palestinian', 0.03717220165789062), ('jewish', 0.030863875710338357), ('jew', 0.019192169334237005), ('state', 0.015849017256660238), ('arab', 0.015275533079610031), ('jerusalem', 0.011977999061571347), ('peace', 0.01170429070434284), ('gaza', 0.011508784734893905)]


0.16876267484034746 [('attack', 0.03117726812323578), ('killed', 0.015107591580645263), ('people', 0.01468211506282129), ('terrorist', 0.012954807408371728), ('police', 0.01273889395156553), ('death', 0.009776434316268762), ('report', 0.008484128773325798), ('security', 

# Below This Is Katie Sketching Out Comparison Stuff

In [18]:
K = ldamodel_10k.num_topics
print(K)

100


In [19]:
ldamodel_10k.num_topics

100

In [35]:
THRESH = 0.1
all_untrusted_tops = ldamodel_10k[doc_term_matrix_unshuffled]

In [174]:
# documents with less than 100 topics in all data
# trusted, untrusted: 848, 4420
messed_up_docs = []
trusted_bad = None
for i,t in enumerate(all_untrusted_tops):
    if len(t) != 100:
        messed_up_docs.append(i)
    # trusted docs end here
    if i == 19999:
        trusted_bad = len(messed_up_docs)
new_total_len = 70000 - len(messed_up_docs)
new_trusted_len = 20000 - trusted_bad
trusted_bad, len(messed_up_docs), new_trusted_len, new_total_len, messed_up_docs[:15]

(848,
 4420,
 19152,
 65580,
 [5, 14, 15, 45, 59, 98, 124, 147, 182, 216, 237, 291, 345, 360, 374])

In [180]:
bad_array = [(-1,-1) for i in range(100)]
all_untrusted_tops_clean = [bad_array if i in messed_up_docs else t for i,t in enumerate(all_untrusted_tops)]
train_docs_unshuffled_clean = ["-1" if i in messed_up_docs else t for i,t in enumerate(train_docs_unshuffled)]

In [183]:
len(all_untrusted_tops_clean[15])

100

In [184]:
# Unshuffled Matrix => 0-20k -> trustworthy; 20k-70k -> untrustworthy
THRESH = 0.1
all_trusted_tops = all_untrusted_tops_clean[:20000]
all_trusted_tops_lines = list(map(lambda x: [round(v, 3) if v > THRESH else 0 for t, v in x], all_trusted_tops))

In [185]:
print(len(all_trusted_tops[15]))
print(all_trusted_tops_lines[15])

100
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [95]:
tops = all_trusted_tops[7090]
tops = sorted(tops, key=lambda a:a[1], reverse=True)
for top in tops[:10]:
    print(top[0])
    print(top[1], ldamodel_10k.show_topic(top[0]))
    print("\n")

52
0.7246350682590614 [('syria', 0.04086777070932751), ('syrian', 0.027469172650214042), ('iraq', 0.01857656950863848), ('isi', 0.018187709992666248), ('state', 0.015005201848788225), ('force', 0.014193372332986544), ('group', 0.011577098221138267), ('terrorist', 0.010884655398836832), ('assad', 0.01062882676990773), ('government', 0.010113758463663807)]


0
0.09307309510871009 [('military', 0.06252925938641116), ('force', 0.036800505602197185), ('army', 0.021121375737336537), ('defense', 0.017610249367997252), ('troop', 0.015803969913548266), ('afghanistan', 0.015534783558565588), ('soldier', 0.013962579195405887), ('security', 0.011153678099934458), ('operation', 0.009347398645485472), ('iraq', 0.008805124683998626)]


42
0.06850161790574973 [('photo', 0.0515457446252027), ('image', 0.03772558455824659), ('castro', 0.02398911963174138), ('cuban', 0.020850551864832347), ('cuba', 0.019197572840926925), ('playing', 0.01511743474394518), ('fidel', 0.011790552911021605), ('le', 0.01055604

In [70]:
######## Feel free to look at the output
######## Uncomment whatever you want to see

# UNSHUFFLED_DOC = 100

# # prints doc
# print(train_docs_unshuffled[UNSHUFFLED_DOC])

# # prints english topics
# topics = ldamodel_10k[doc_term_matrix_unshuffled[UNSHUFFLED_DOC]]
# tops = sorted(topics, key=lambda a:a[1], reverse=True)
# for top in tops[:10]:
#     print(top[1], ldamodel_10k.show_topic(top[0]))
#     print("\n")
    
# # prints wihtout rounding
# print(all_trusted_tops[UNSHUFFLED_DOC])

# # prints with rounding as is written as a row of csv
# print(all_trusted_tops_lines[UNSHUFFLED_DOC])

In [186]:
with open('Matrices/topic_doc_matrix_clean.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(all_trusted_tops_lines)

In [ ]:
import gensim.matutils as mt

In [ ]:
term_topic_matrix = ldamodel_10k.get_topics()
K = 100
topic_comps = np.zeros((K,K))
ind = list(range(term_topic_matrix.shape[1]))
for i1 in range(K):
    print('!!!' + str(i1))
    a = list(term_topic_matrix[i1, :])
    #print('A')
    a = list(zip(ind, a))
    #print('B')
    #a = [(1,4)]
    #print(a)
    for j1 in range(i1, K):
        #print(j1)
        b = list(term_topic_matrix[j1, :])
        #print('a')
        b = list(zip(ind, b))
        #print('b')
        sim = mt.cossim(a, b)
        #print('c')
        topic_comps[i1, j1] = sim
        #print('d')
        topic_comps[j1, i1] = sim  
        #print('e')

In [ ]:
#np.savetxt("topic_sims.csv", topic_comps, delimiter=",")

In [44]:
def get_docs(top, matches):
    tot_length = 0
    docs = []
    scores = []
    print("NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN")
    print(top)
    while(tot_length < 500 and len(docs) < 3):
        doc_i = matches.loc[top, 'match_' + str(len(docs) + 1)]
        print(doc_i)
        doc = train_docs_unshuffled[doc_i]
        b = ldamodel_10k[doc_term_matrix_unshuffled[doc_i]]
        print(sorted(b, key=lambda a:a[1], reverse=True))
        scores.append(matches.loc[top, 'value_' + str(len(docs) + 1)])
        tot_length += len(doc.split()) 
        docs.append(doc)
    return docs, scores

In [33]:
#Do sampling for documents to make summary for
nums = [random.randint(0, 9999) for i in range(5*30)]
type_nums = [10000*(int(i/30)+2) for i in range(30*5)]
sampled_doc_nums = list(tuple(map(sum, zip(nums, type_nums))))
sampled_doc_nums = sorted(sampled_doc_nums)
# Unshuffled Matrix => 0-20k -> trustworthy; 20k-70k -> untrustworthy

with open('sampled_tr_docs_10k_not_shuffled.json', 'r') as outfile:  
    train_docs_unshuffled = json.load(outfile)
matches = pd.read_csv('Matrices/topics_match_indices_2.csv')
print(nums)
print(type_nums)
print(sampled_doc_nums)

[7881, 3626, 8661, 1168, 605, 8775, 4821, 460, 4088, 2876, 5018, 8894, 5249, 5064, 8652, 601, 58, 1637, 5480, 3227, 4079, 4770, 9201, 7073, 226, 5147, 7869, 7806, 4207, 1647, 4218, 2518, 6879, 7014, 5433, 1657, 3266, 1421, 4316, 2015, 8691, 357, 8505, 2648, 6019, 9572, 4544, 9723, 8136, 6527, 653, 9012, 5211, 2731, 7761, 6877, 4907, 8564, 6245, 3070, 2443, 8129, 1899, 8758, 4421, 2659, 5383, 2116, 7143, 6827, 2327, 7561, 7922, 2532, 2425, 1606, 5219, 4750, 8810, 9991, 8559, 5291, 2885, 8692, 1166, 1557, 7017, 5932, 6028, 7056, 8476, 1591, 77, 4640, 2832, 1467, 7912, 4216, 1795, 2841, 6870, 6051, 3475, 4705, 63, 6692, 6240, 6035, 3476, 8781, 9441, 3371, 4068, 81, 9898, 7613, 8280, 9309, 3355, 5983, 8937, 2288, 4441, 4813, 2511, 9069, 3278, 7094, 2822, 3493, 896, 8413, 90, 7352, 2604, 8246, 3088, 5189, 5554, 2823, 3845, 4895, 3878, 867, 3465, 5664, 1585, 8629, 6479, 1803]
[20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 2000

In [188]:
for i in sampled_doc_nums:
    if i in messed_up_docs:
        print(i)

21168
28661
34218
42327
48129
56051
58476
65554
68413
69069


In [41]:
print(matches)

    index  value_1  value_2  value_3  match_1  match_2  match_3
0       0    0.770    0.767    0.752    11014     6274    13670
1       1    0.860    0.776    0.686    19921    12480      311
2       2    0.582    0.483    0.482     8986     8710     3106
3       3    0.744    0.740    0.738    14571     7612    17087
4       4    0.994    0.994    0.993    11115    11115      116
5       5    0.994    0.994    0.989    13810    13810    16262
6       6    0.970    0.365    0.322     6132    14462     8641
7       7    0.791    0.699    0.576    14148    15342     5758
8       8    0.969    0.934    0.921    18645    12695     6955
9       9    0.884    0.854    0.836    19226    18610    12531
10     10    0.218    0.207    0.184     8062     4321     6874
11     11    0.611    0.602    0.572    14646    14269    19729
12     12    0.632    0.622    0.564    18657     6937     8904
13     13    0.498    0.370    0.352     1569     4830    11161
14     14    0.729    0.604    0.502    

In [50]:
print(all_untrusted_tops[:5])

[[(0, 0.0022617167428406575), (1, 0.00022975638196883732), (2, 3.554721633405734e-05), (3, 1.1625804748901216e-05), (4, 0.00019399540815709035), (5, 0.00010095436987704412), (6, 9.282507899210898e-05), (7, 4.613188546568354e-05), (8, 6.891778752179942e-05), (9, 0.00017892835894694787), (10, 9.615135433182658e-05), (11, 0.0016114112060415973), (12, 0.00013907438493430033), (13, 0.0934878397143368), (14, 0.00035937785174711676), (15, 5.28505158569313e-05), (16, 0.0007525561573610998), (17, 0.013104783557516831), (18, 0.00036641918922359425), (19, 0.00010243188276656357), (20, 1.4055849832434545e-05), (21, 0.00021650441322635918), (22, 0.0020275468524839175), (23, 3.9436409699402706e-05), (24, 0.00020486782435606235), (25, 1.3971006970125317e-05), (26, 0.0003154245877369151), (27, 7.907435385474066e-06), (28, 0.0001758483143601364), (29, 0.06362485242276358), (30, 0.00019326108764138454), (31, 0.00013464128791982694), (32, 1.4092024206660287e-05), (33, 6.178135370675756e-05), (34, 1.08849

In [36]:
reduce_all_untrusted_tops = list(filter(lambda x: x[0] in sampled_doc_nums, list(enumerate(all_untrusted_tops))))
sampled_untrusted_tops = [i[1] for i  in reduce_all_untrusted_tops]
sampled_untrusted_tops_lines = list(map(lambda x: [round(v, 3) if v > THRESH else 0 for t, v in x], sampled_untrusted_tops))

In [53]:
len(sampled_doc_nums)

150

In [38]:
len(reduce_all_untrusted_tops), len(sampled_untrusted_tops), len(sampled_untrusted_tops_lines)

(150, 150, 150)

In [43]:
print(matches.loc[])

index         51.000
value_1        0.688
value_2        0.675
value_3        0.597
match_1     1480.000
match_2    16536.000
match_3     8624.000
Name: 51, dtype: float64


In [85]:
SUM_THRESH = .15
doc_matches_matrix = []
num = 0
for doc in sampled_untrusted_tops_lines:
    print(doc)
    tops = all_untrusted_tops[sampled_doc_nums[num]]
    tops = sorted(tops, key=lambda a:a[1], reverse=True)
    print(tops)
    matched_documents = [train_docs_unshuffled[sampled_doc_nums[num]]]
    print(sampled_doc_nums[num])
    for i in range(len(doc)):
        top = doc[i]
        if(top < SUM_THRESH):
            continue
        #print(top)
        #print(ldamodel_10k.show_topic(i))
        topic_docs, scores = get_docs(i, matches)
        matched_documents.append({'documents': topic_docs, 'scores': scores})
    with open('matches2/' + str(sampled_doc_nums[num]) +'.json', 'w') as fout:
        json.dump(matched_documents, fout)
    num += 1

[0, 0.147, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.181, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.363, 0, 0.178, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[(67, 0.36334238617483344), (51, 0.18115449937444478), (69, 0.17837401966567382), (1, 0.14694384920972145), (94, 0.040619352216180585), (75, 0.02491115182334902), (0, 0.02363335015734314), (56, 0.009749873308810128), (61, 0.0048196822775405), (95, 0.0037509220241560555), (83, 0.00294734584929379), (43, 0.002808515334356675), (76, 0.0024274109389617977), (38, 0.0019508782945772153), (66, 0.001406357506427415), (39, 0.0011292054418769004), (12, 0.001112369440356464), (74, 0.0008296161500188044), (89, 0.0007231059327042062), (47, 0.0006620578873451579), (21, 0.0006132040445629856), (79, 0.0003541819700775311), (37, 0.0003473996448634314), (46, 0.00034646209238420225), (29, 0.00

KeyboardInterrupt: 

In [ ]:
sampled_untrusted_tops[2] == all_untrusted_tops[sampled_doc_nums[2]]

In [ ]:
a = all_untrusted_tops[sampled_doc_nums[20]]
sorted(a, key=lambda a:a[1], reverse=True)

In [84]:
b = ldamodel_10k[doc_term_matrix_unshuffled[1481]]
sorted(b, key=lambda a:a[1], reverse=True)

[(58, 0.27226377441171057),
 (13, 0.1291358416616669),
 (67, 0.11849222180165092),
 (56, 0.09572316367970782),
 (96, 0.08738602712607557),
 (90, 0.0655666609253252),
 (54, 0.06119664963357997),
 (47, 0.023675741245974703),
 (12, 0.022400562309704074),
 (4, 0.02119155034074737),
 (70, 0.019834566103028153),
 (37, 0.01647892069672425),
 (78, 0.009630584509046133),
 (79, 0.009319631792036672),
 (11, 0.008100775076142746),
 (6, 0.008061178651553843),
 (22, 0.005411768809463271),
 (98, 0.0037599948141963345),
 (28, 0.003539571530994863),
 (17, 0.0030078789750283908),
 (89, 0.00292365492035145),
 (60, 0.001266551451308764),
 (94, 0.0011375544943288693),
 (15, 0.0010618960591803226),
 (51, 0.000983240117767941),
 (43, 0.0009809240087466618),
 (14, 0.0006162475750438423),
 (26, 0.0006049871328341087),
 (31, 0.0005586720309658429),
 (97, 0.0004497725779135512),
 (46, 0.0004035062455721024),
 (39, 0.0003961990756308122),
 (55, 0.0003833100169889714),
 (16, 0.00036752434872546657),
 (36, 0.000231

## all_untrusted_tops[sampled_doc_nums[num]]

In [ ]:
###Summarizing done in a separate script